In [85]:
import dev.biserman.planet.language.Language
import dev.biserman.planet.language.Segment
import dev.biserman.planet.language.SyllableConstructor
import dev.biserman.planet.language.InventoryTransformation
import dev.biserman.planet.language.Manner
import dev.biserman.planet.language.SegmentType


In [86]:
import dev.biserman.planet.language.Glide
import dev.biserman.planet.language.Place
import dev.biserman.planet.language.SegmentData
import dev.biserman.planet.utils.toWeightedBag
import kotlin.random.Random

SyllableConstructor.languageFile = """E:\Users\Winggar\source\repos\Planet\planet\english.json"""
SyllableConstructor.phonemeFile = """E:\Users\Winggar\source\repos\Planet\planet\phonemes.json"""

val basicPhonemes = "ptksmnljw"
val basicTransformation: InventoryTransformation = { inventory ->
    inventory.plus(basicPhonemes.mapNotNull { SyllableConstructor.segments[it.toString()] })
}

fun (Set<Segment>).addSet(
    from: (SegmentData) -> Boolean,
    to: (SegmentData) -> SegmentData,
    condition: Boolean = true
): Set<Segment> {
    if (!condition) return this

    val new = this
        .filter { from(it.data) }
        .map { to(it.data) }
    val matching = SyllableConstructor.segments.values.filter { it.data in new }
    return this.plus(matching)
}

val random = Random(System.currentTimeMillis())

val placeWeights = mapOf(
    Place.BILABIAL to 45,
    Place.LABIODENTAL to 35,
    Place.DENTAL to 5,
    Place.ALVEOLAR to 50,
    Place.POSTALVEOLAR to 35,
    Place.PALATAL to 25,
    Place.VELAR to 45,
    Place.UVULAR to 5,
    Place.GLOTTAL to 50,
)

val inversePlaceWeights = (placeWeights.values.max()).let { maxWeight ->
    placeWeights.mapValues { (_, weight) -> maxWeight - weight + 1 }
}

//val affricates = mapOf(
//    'ɸ' to Pair('p', 10),
//    'β' to Pair('b', 10),
//    'f' to Pair('p', 50),
//    'v' to Pair('b', 50),
//    'θ' to Pair('t', 10),
//    'ð' to Pair('d', 10),
//    's' to Pair('t', 100),
//    'z' to Pair('d', 100),
//    'ʃ' to Pair('t', 250),
//    'ʒ' to Pair('d', 250),
//    'x' to Pair('k', 100)
//)

fun center(x: Double) = 4 * (x - 0.5).pow(3) + 0.5

val inventoryTransformations = listOf<Pair<InventoryTransformation, Int>>(
    { inventory: Set<Segment> -> // add voicing
        val chance = random.nextDouble()
        val newInventory = inventory.let {
            if (chance <= 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.PLOSIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }.let {
            if (chance <= 0.4 || chance > 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.FRICATIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }
        newInventory
    } to 600,
    { inventory: Set<Segment> -> // add aspiration
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isAspirated = true, voiced = null) },
            condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
    } to 400,
    { inventory: Set<Segment> -> // add ejectives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isEjective = true, voiced = null) })
    } to 100,

    { inventory: Set<Segment> -> // add plosives from fricatives
        inventory.addSet(
            from = { it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.PLOSIVE) })
    } to 300,
    { inventory: Set<Segment> -> // add fricatives from plosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE },
            to = { it.copy(manner = Manner.FRICATIVE) })
    } to 300,

    { inventory: Set<Segment> -> // add rhotic
        inventory.plus(SyllableConstructor.segments["ɹ"]!!)
    } to 500,
    { inventory: Set<Segment> -> // add implosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.IMPLOSIVE) })
    } to 100,
    { inventory: Set<Segment> -> // add clicks
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.CLICK, voiced = null) })
    } to 100,
    { inventory: Set<Segment> -> // add nasals
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.NASAL, voiced = null) })
    } to 250,
    { inventory: Set<Segment> -> // remove randomly
        inventory.filter { random.nextDouble() <= center(it.prevalence).pow(0.33) }.toSet()
    } to 500,

    affricate@{ inventory: Set<Segment> -> // add affricates
        val fricative =
            inventory.filter { it.data.manner == Manner.FRICATIVE && it.data.place != Place.GLOTTAL }
                .randomOrNull(random) ?: return@affricate inventory
        val glide = Glide.from(fricative.data)
        val affricate = if (random.nextDouble() <= 0.75) {
            val place = when (fricative.data.place) {
                Place.LABIODENTAL -> Place.BILABIAL
                Place.POSTALVEOLAR -> Place.ALVEOLAR
                Place.PALATAL -> Place.ALVEOLAR
                Place.DENTAL -> Place.ALVEOLAR
                else -> fricative.data.place
            }
            fricative.copyData { it.copy(place = place, manner = Manner.PLOSIVE, consonantGlide = glide) }
        } else {
            val plosive =
                inventory
                    .filter {
                        it.data.manner == Manner.PLOSIVE &&
                                it.data.place != Place.GLOTTAL &&
                                it.data.isAspirated == false &&
                                it.data.isEjective == false &&
                                it.data.voiced == fricative.data.voiced
                    }
                    .randomOrNull(random) ?: return@affricate inventory
            plosive.copyData { it.copy(consonantGlide = glide) }
        }
        inventory.plus(affricate)
    } to 300,

    glideColoredSet@{ inventory: Set<Segment> -> // add a glide-colored set
        val chance = random.nextDouble()
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSet inventory

        val newInventory = inventory.let {
            if (chance <= 0.67) {
                it.plus(it.filter { it.data.manner == Manner.FRICATIVE }
                    .map { it.copyData { it.copy(consonantGlide = Glide.from(glide.data)) } })
            } else it
        }.let {
            if (chance <= 0.33 || chance > 0.67) {
                it.plus(it.filter { it.data.manner == Manner.PLOSIVE }
                    .filter { it.data.place != Place.GLOTTAL }
                    .map { it.copyData { it.copy(consonantGlide = Glide.from(glide.data)) } })
            } else it
        }

        newInventory
    } to 200,

    glideColoredSingle@{ inventory: Set<Segment> ->
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        val plosive = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.PLOSIVE }
            .filter { it.data.place != Place.GLOTTAL }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        inventory.plus(plosive.copyData { it.copy(consonantGlide = Glide.from(glide.data)) })
    } to 300,

    { inventory: Set<Segment> -> // tʃ, dʒ
        if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE }) {
            val tʃ = SyllableConstructor.segments["t"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE
                    )
                )
            }
            val dʒ = SyllableConstructor.segments["d"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE
                    )
                )
            }

            if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE && it.data.voiced == true }) {
                inventory.plus(listOf(tʃ, dʒ))
            } else {
                inventory.plus(tʃ)
            }
        } else {
            inventory
        }
    } to 200,
).plus(placeWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.addSet(
            from = { it.type == SegmentType.CONSONANT },
            to = { it.copy(place = place) })

    } to weight * 3
}).plus(inversePlaceWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.filter { it.data.place != place }.toSet()
    } to weight
})

val bag = inventoryTransformations.toWeightedBag(random) { it.second }

fun generateConsonants() = basicTransformation(setOf()).let {
    (1..15).fold(it) { acc, _ -> bag.grab()!!.first.invoke(acc) }
}.sortedBy { SyllableConstructor.segments.keys.indexOf(it.symbol) }

for (_1 in 0..10) {
    val testLanguage = generateConsonants()
    println("${testLanguage.size} phonemes: ${testLanguage.map { it.display }}")
}


22 phonemes: [m, n, p, pf, pl, t, tʃ, k, kl, b, d, dʒ, ɡ, f, s, v, z, ʔ, h, w, ɹ, l]
16 phonemes: [n, ŋ, p, pw, t, tw, k, kw, s, sw, z, zw, j, w, ɹ, l]
8 phonemes: [m, n, t, k, b, d, ɹ, l]
19 phonemes: [m, n, ŋ, p, t, tʃ, tj, ts, k, b, d, ɡ, f, s, v, z, j, w, l]
18 phonemes: [m, n, ŋ, p, pf, t, k, b, d, ɡ, f, s, v, z, h, w, ɹ, l]
15 phonemes: [m, n, ŋ, p, pɹ, pf, t, tɹ, k, kɹ, f, s, w, ɹ, l]
16 phonemes: [m, n, p, pl, pf, t, tʃ, tɹ, ts, k, f, s, j, w, ɹ, l]
15 phonemes: [m, n, ŋ, p, t, k, b, d, ɡ, s, z, j, w, ɹ, l]
20 phonemes: [m, n, ŋ, p, t, tʃ, k, b, d, dʒ, ɡ, s, sl, ʃ, z, zl, ʒ, j, w, l]
16 phonemes: [m, n, p, t, ts, b, d, s, ʃ, z, ʒ, ʔ, h, j, w, l]
16 phonemes: [m, n, p, t, ts, tʃ, k, kw, f, s, v, z, j, w, ɹ, l]


In [87]:
SyllableConstructor.segments.entries.map {
    println("${it.key} - ${it.value}")
}

m - Segment(symbol=m, data=SegmentData(type=CONSONANT, place=BILABIAL, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.97)
n - Segment(symbol=n, data=SegmentData(type=CONSONANT, place=ALVEOLAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.97)
ŋ - Segment(symbol=ŋ, data=SegmentData(type=CONSONANT, place=VELAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.69)
p - Segment(symbol=p, data=SegmentData(type=CONSONANT, place=BILABIAL, manner=PLOSIVE, voiced=false, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.95)
t - Segment(symbol=t, data=Segme

[kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit]

In [88]:
SyllableConstructor.glideMap

{(Glide(place=LABIODENTAL, manner=FRICATIVE), false)=f, (Glide(place=DENTAL, manner=FRICATIVE), false)=θ, (Glide(place=ALVEOLAR, manner=FRICATIVE), false)=s, (Glide(place=POSTALVEOLAR, manner=FRICATIVE), false)=ʃ, (Glide(place=LABIODENTAL, manner=FRICATIVE), true)=v, (Glide(place=ALVEOLAR, manner=FRICATIVE), true)=z, (Glide(place=POSTALVEOLAR, manner=FRICATIVE), true)=ʒ, (Glide(place=PALATAL, manner=SEMIVOWEL), null)=j, (Glide(place=LABIOVELAR, manner=SEMIVOWEL), null)=w, (Glide(place=RETROFLEX, manner=LIQUID), null)=ɹ, (Glide(place=ALVEOLAR, manner=LIQUID), null)=l}

In [89]:
fun generateVowels(): List<Segment> {
    val allVowels = SyllableConstructor.segments.values
        .filter { it.data.type == SegmentType.VOWEL }

    val baseVowels = allVowels
        .filter { random.nextDouble() <= it.prevalence }

    val nasalVowels = if (random.nextDouble() <= 0.25) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.5 || (it in baseVowels && chance <= it.prevalence * 1.5)
        }.map { it.copy(symbol = it.symbol + '̃') }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.05 }
        .map { it.copy(symbol = it.symbol + '̃') }

    val longVowels = if (random.nextDouble() <= 0.5) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.25 || (it in baseVowels && chance <= it.prevalence * 2.0)
        }.map { it.copy(symbol = it.symbol + 'ː') }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.1 }
        .map { it.copy(symbol = it.symbol + 'ː') }

    val wOnGlides = if (random.nextDouble() <= 0.02) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.25 || (it in baseVowels && chance <= it.prevalence * 2.0)
        }.filter { it.symbol != "u" }.map { it.copy(symbol = 'u' + it.symbol) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "u" }.map { it.copy(symbol = 'u' + it.symbol) }

    val jOnGlides = if (random.nextDouble() <= 0.02) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.25 || (it in baseVowels && chance <= it.prevalence * 2.0)
        }.filter { it.symbol != "i" }.map { it.copy(symbol = 'i' + it.symbol) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "i" }.map { it.copy(symbol = 'i' + it.symbol) }

    val wOffGlides = if (random.nextDouble() <= 0.02) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.25 || (it in baseVowels && chance <= it.prevalence * 2.0)
        }.filter { it.symbol != "o" }.map { it.copy(symbol = it.symbol + 'o') }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "o" }.map { it.copy(symbol = it.symbol + 'o') }

    val jOffGlides = if (random.nextDouble() <= 0.02) {
        allVowels.filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 0.25 || (it in baseVowels && chance <= it.prevalence * 2.0)
        }.filter { it.symbol != "i" }.map { it.copy(symbol = it.symbol + 'i') }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "i" }.map { it.copy(symbol = it.symbol + 'i') }

    return baseVowels + nasalVowels + longVowels + wOnGlides + jOnGlides + wOffGlides + jOffGlides
}

for (_1 in 0..10) {
    println(generateVowels().map { it.display })
}


[i, u, e, o, ɛ, a, ĩ, ũ, ẽ, õ, ɛ̃, ã, iː, uː, eː, oː, ɛː, aː]
[i, u, ɪ, e, a, uɛ]
[i, u, e, o, æ, a, iː, uː, eː, oː, aː]
[i, u, e, o, a, ũ, uː, ɛː, ie]
[i, u, e, o, ɛ, a]
[i, u, e, o, ɛ, a, ɪ̃, iː, uː, eː, oː, ɛː, aː]
[i, u, ɪ, e, o, a, iː, oː, ua]
[i, u, ɪ, e, ɛ, a, ũ, ẽ, iː, uː, eː, ɛː, aː]
[i, u, o, a, ĩ, ũ, õ, ã, iː, uː, eː, oː, aː]
[i, u, e, ɛ, a]
[i, u, e, o, ɛ, a, ĩ, ũ, ẽ, õ, ɛ̃, ã, iː, uː, eː, oː, ɛː, aː]


In [90]:
val allGlides = SyllableConstructor.segments.values
    .filter { it.data.place != Place.GLOTTAL }
    .filter {
        it.data.manner in listOf(
            Manner.FRICATIVE,
            Manner.SEMIVOWEL,
            Manner.LIQUID
        )
    }.map { Glide.from(it.data) }.toSet().plus(null)

class ConsonantSlot(
    val manner: Set<Manner> = Manner.values().toSet(),
    val place: Set<Place> = Place.values().toSet(),
    val voiced: Set<Boolean?> = setOf(true, false, null),
    val consonantGlide: Set<Glide?> = allGlides
) {
    fun getMatching(segments: List<Segment>) = segments.filter {
        it.data.type == SegmentType.CONSONANT &&
                it.data.manner in manner &&
                it.data.place in place &&
                it.data.voiced in voiced &&
                it.data.consonantGlide in consonantGlide
    }

    companion object
}

In [91]:
fun (ConsonantSlot.Companion).gen() = ConsonantSlot(
    manner = random.nextDouble().let {
        when {
            it < 0.5 -> setOf(Manner.values().random(), Manner.values().random())
            else -> setOf(Manner.values().random())
        }
    },
    place = random.nextDouble().let {
        when {
            it < 0.2 -> setOf(Place.values().random(), Place.values().random())
            it < 0.4 -> Place.values().filter { random.nextDouble() < 0.8 }.toSet()
            else -> Place.values().toSet()
        }
    },
    voiced = random.nextDouble().let {
        when {
            it < 0.15 -> setOf(true, null)
            it < 0.3 -> setOf(false, null)
            else -> setOf(true, false, null)
        }
    },
    consonantGlide = if (random.nextDouble() < 0.7) allGlides
    else allGlides.random()?.manner.let { chosenManner ->
        random.nextDouble().let {
            when {
                it < 0.2 -> allGlides.filter {
                    it?.manner == chosenManner && it?.place in listOf(
                        Place.values().random(),
                        Place.values().random()
                    )
                }
                it < 0.4 -> allGlides.filter { it?.manner == chosenManner && random.nextDouble() < 0.8 }
                else -> allGlides.filter { it?.manner == chosenManner }
            }.toSet()
        }
    }
)

In [92]:
// adapted from: https://gist.github.com/erikhuizinga/d2ca2b501864df219fd7f25e4dd000a4

import kotlin.reflect.KFunction

/**
 * Create the cartesian product of any number of sets of any size. Useful for parameterized tests
 * to generate a large parameter space with little code. Note that any type information is lost, as
 * the returned set contains list of any combination of types in the input set.
 *
 * @param sets The sets.
 */
fun <T> cartesianProduct(vararg sets: Set<T>) =
    sets
        .fold(listOf(listOf<T>())) { acc, set ->
            acc.flatMap { list -> set.map { element -> list + element } }
        }
        .toSet()


In [93]:
fun isSonoritySequenced(cluster: List<ConsonantSlot>, isOnset: Boolean): Boolean = when {
    isOnset -> cluster
        .windowed(size = 2) { (a, b) ->
            a.manner.maxOf { it.ordinal } <= b.manner.minOf { it.ordinal }
        }
        .all { it }
    else -> isSonoritySequenced(cluster.reversed(), !isOnset)
}

typealias Cluster = List<ConsonantSlot>

fun generateClusters(
    onsetMaxConsonants: Int,
    codaMaxConsonants: Int,
    fallOff: Double,
    sonoritySequencingStrictness: Double,
    consonants: List<Segment>
): Pair<List<Cluster>, List<Cluster>> {
    val onsetClusters = mutableListOf<List<ConsonantSlot>>(listOf(ConsonantSlot()))
    val codaClusters = mutableListOf<List<ConsonantSlot>>(listOf())

    for (i in 1..<onsetMaxConsonants) {
        val attempts = (onsetMaxConsonants - i * fallOff + random.nextDouble(1.0, 3.0)).roundToInt()
        for (_j in 1..attempts) {
            while (true) {
                val newCluster = (1..i).map { ConsonantSlot.gen() }
                if (random.nextDouble() < sonoritySequencingStrictness && !isSonoritySequenced(
                        newCluster,
                        true
                    )
                ) continue
                if (newCluster.any { it.getMatching(consonants).isEmpty() }) continue
                onsetClusters.add(newCluster)
                break
            }
        }
    }

    for (i in 0..<codaMaxConsonants) {
        val attempts = (onsetMaxConsonants - i * fallOff + random.nextDouble(-1.0, 1.0)).roundToInt()
        for (_j in 1..attempts) {
            while (true) {
                val newCluster = (1..i).map { ConsonantSlot.gen() }
                if (random.nextDouble() < sonoritySequencingStrictness && !isSonoritySequenced(
                        newCluster,
                        false
                    )
                ) continue
                if (newCluster.any { it.getMatching(consonants).isEmpty() }) continue
                codaClusters.add(newCluster)
                break
            }
        }
    }

    return onsetClusters to codaClusters
}

class Syllable(val onset: List<Segment>, val nucleus: List<Segment>, val coda: List<Segment>) {
    val allConsonants = onset + coda
    override fun toString() =
        "${onset.joinToString("") { it.display }}${nucleus.joinToString("") { it.display }}${coda.joinToString("") { it.display }}"
}


In [94]:
import dev.biserman.planet.language.SyllableConstructor

class SyllableRule(val name: String, val check: (Syllable) -> Boolean)

fun generateSyllableRules(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    maxOnset: Int,
    maxCoda: Int
): List<SyllableRule> {
    val rules = mutableListOf<SyllableRule>()

    class SidedRule(val segment: Segment, val allowedInOnset: Boolean, val allowedInCoda: Boolean)

    val possibleOnsets = onsetClusters.flatMap { cluster -> cluster.flatMap { it.getMatching(consonants) } }
    val possibleCodas = codaClusters.flatMap { cluster -> cluster.flatMap { it.getMatching(consonants) } }

    if (maxOnset > 0 && maxCoda > 0) {
        val isRareRestrictive = random.nextDouble() < 0.33
        val sidedPhonemeRules = consonants.map { consonant ->
            when (consonant.display[0]) {
                in "ŋɳ" -> random.nextDouble().let {
                    when {
                        it >= 0.7 && consonant in possibleCodas -> SidedRule(consonant, false, true)
                        it >= 0.65 && consonant in possibleOnsets -> SidedRule(consonant, true, false)
                        else -> SidedRule(consonant, true, true)
                    }
                }
                in "ptkmnlr" -> random.nextDouble().let {
                    when {
                        it >= 0.97 && consonant in possibleCodas -> SidedRule(consonant, false, true)
                        it >= 0.7 && consonant in possibleOnsets -> SidedRule(consonant, true, false)
                        else -> SidedRule(consonant, true, true)
                    }
                }
                else -> random.nextDouble().let {
                    when {
                        it >= 0.95 && consonant in possibleCodas-> SidedRule(consonant, false, true)
                        it >= 0.7 || isRareRestrictive && consonant in possibleOnsets -> SidedRule(consonant, true, false)
                        else -> SidedRule(consonant, true, true)
                    }
                }
            }
        }.filter { !it.allowedInOnset || !it.allowedInCoda }.map { rule ->
            SyllableRule("${rule.segment.display} Sidedness (${rule.allowedInOnset}-${rule.allowedInCoda})") { rule.segment !in it.onset || rule.allowedInOnset && rule.segment !in it.coda || rule.allowedInCoda }
        }
        rules.addAll(sidedPhonemeRules)
    }

    if (random.nextDouble() <= 0.99 && maxOnset > 1 || maxCoda > 1) {
        rules.add(SyllableRule("Offglide-Glide Adjacency") { syllable ->
            listOf(syllable.onset, syllable.coda).all { cluster ->
                cluster.windowed(size = 2) { (a, b) -> a.data.offGlide == null || SyllableConstructor.glideMap[a.data.offGlide!! to null] != b.symbol }
                    .all { it }
            }
        })
    }

    if (random.nextDouble() <= 0.8 && consonants.any { it.data.manner == Manner.SEMIVOWEL }) {
        rules.add(SyllableRule("Semivowel-Vowel Adjacency") { syllable ->
            (syllable.onset.last().symbol != "w" || syllable.nucleus.first().symbol != "u") &&
                    (syllable.onset.last().symbol != "j" || syllable.nucleus.first().symbol != "i")
        })
    }

    while (random.nextDouble() <= 0.33) {
        val place = listOf(Syllable::onset, Syllable::coda, Syllable::allConsonants).random()
        val consonantFeature = listOf(SegmentData::place, SegmentData::manner, SegmentData::voiced).random()
        val vowelFeature = listOf(SegmentData::depth, SegmentData::height).random()
        val modelConsonant = consonants.random()
        val modelVowel = vowels.random()

        rules.add(
            SyllableRule(
                "${consonantFeature.get(modelConsonant.data)} consonants cannot co-occur with ${
                    vowelFeature.get(modelVowel.data)
                } vowels in ${place.name}"
            ) { syllable ->
                place.get(syllable)
                    .none { consonant -> consonantFeature.get(consonant.data) == consonantFeature.get(modelConsonant.data) } ||
                        syllable.nucleus.none { vowel -> vowelFeature.get(vowel.data) == vowelFeature.get(modelVowel.data) }
            })
    }

    if (maxOnset > 1 || maxCoda > 1) {
        // needs probability analysis
        if (random.nextDouble() <= 0.85) {
            rules.add(SyllableRule("Homorganic Consonant Voicing") { syllable ->
                listOf(syllable.onset, syllable.coda).all { cluster ->
                    cluster.windowed(size = 2) { (a, b) -> a.data.voiced == b.data.voiced || a.data.voiced == null || b.data.voiced == null }
                        .all { it }
                }
            })
        }

        // needs probability analysis
        while (random.nextDouble() <= 0.75) {
            val manners = consonants.filter { it.data.manner != Manner.LIQUID }
                .sortedBy { random.nextDouble() }
                .take(2)
                .map { it.data.manner }

            rules.add(SyllableRule("Homorganic ${manners[0]!!.name}-${manners[1]!!.name} Consonant Placing") { syllable ->
                listOf(syllable.onset, syllable.coda).all { cluster ->
                    cluster.windowed(size = 2) { (a, b) -> a.data.place == b.data.place || a.data.manner !in manners || b.data.manner !in manners }
                        .all { it }
                }
            })
        }
    }

    return rules.distinctBy { it.name }
}


In [95]:
fun generateSyllable(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    syllableRules: List<SyllableRule> = listOf()
): Syllable {
    while (true) {
        val onset = onsetClusters.random()
        val nucleus = vowels.random()
        val coda = codaClusters.random()

        val syllable = Syllable(
            onset.map { it.getMatching(consonants).random() },
            listOf(nucleus),
            coda.map { it.getMatching(consonants).random() }
        )

        if (syllableRules.all { it.check(syllable) }) return syllable
    }
}

fun generateAllSyllables(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    syllableRules: List<SyllableRule>
): List<Syllable> {
    val allOnsetClusters = onsetClusters.flatMap { cluster ->
        cartesianProduct(*cluster.map { slot ->
            slot.getMatching(consonants)
                .toSet()
        }.toTypedArray())
    }.toSet()

    val allCodaClusters = codaClusters.flatMap { cluster ->
        cartesianProduct(*cluster.map { slot ->
            slot.getMatching(consonants)
                .toSet()
        }.toTypedArray())
    }.toSet()

    return cartesianProduct(
        allOnsetClusters,
        vowels.map { listOf(it) }.toSet(),
        allCodaClusters
    ).map { (onset, nucleus, coda) ->
        Syllable(onset, nucleus, coda)
    }.filter { syllable -> syllableRules.all { it.check(syllable) } }
}

In [96]:
import dev.biserman.planet.utils.UtilityExtensions.formatDigits
import kotlin.random.nextInt

val consonants = generateConsonants()
println("consonants: ${consonants.map { it.display }}")

val vowels = generateVowels()
println("vowels: ${vowels.map { it.display }}")

val onsetMaxConsonants = random.nextInt(1..3)
val codaMaxConsonants = onsetMaxConsonants + random.nextInt(-1..1)
println("max onset consonants: $onsetMaxConsonants, coda consonants: $codaMaxConsonants")

val fallOff = random.nextDouble()
println("fall off: $fallOff")

val sonoritySequencingStrictness = random.nextDouble(0.0, 1.0).pow(0.5)
println("sonority sequencing strictness: ${sonoritySequencingStrictness.formatDigits()}")

val (onsetClusters, codaClusters) = generateClusters(
    onsetMaxConsonants,
    codaMaxConsonants,
    fallOff,
    sonoritySequencingStrictness,
    consonants
)
println("onset patterns: ${onsetClusters.size}, coda patterns: ${codaClusters.size}")

val syllableRules = generateSyllableRules(consonants, vowels, onsetMaxConsonants, codaMaxConsonants)
println("syllable rules:\n${syllableRules.joinToString("") { " - ${it.name}\n" }}")


val allSyllables = generateAllSyllables(consonants, vowels, onsetClusters, codaClusters, syllableRules)

val trueOnsets = allSyllables.map { it.onset }.distinct()
val trueCodas = allSyllables.map { it.coda }.distinct()

println("onsets (${trueOnsets.size}): ${trueOnsets.map { it.joinToString("") { it.display } }}")
println("codas (${trueCodas.size}): ${trueCodas.map { it.joinToString("") { it.display } }}")

println(
    "sample syllables: ${
        (1..20).map { allSyllables.random().toString() }.joinToString()
    }"
)

println("total possible syllables (${allSyllables.size}): ${allSyllables.map { it.toString() }.sorted().joinToString()}")


org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[96], line 33, column 90: Type mismatch: inferred type is List<Line_69_jupyter.SyllableRule> but List<Line_97_jupyter.SyllableRule> was expected